In [3]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

2024-11-05 13:41:34.325223: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 13:41:34.558647: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-05 13:41:37.640524: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-05 13:41:39.384670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730810500.977939 2655030 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730810501.45

In [4]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [28]:
# Setting these as global variables so that they can be shuffled to ensure an even distribution- many times I wasn't getting all 24/720 labels in the validation set

images = np.load('75/images.npy')
images = images.astype('float32')

# Function that shuffles the images and labels into training, validation, and test sets
def initialize():
    global imgs, train_imgs, val_imgs, test_imgs, train_labels, val_labels, test_labels, distributed
    indices = np.random.permutation(images.shape[0])
    imgs = images[indices]

    split_1 = int(18000*0.7)
    split_2 = int(18000*0.9)

    train_imgs = imgs[:split_1]
    val_imgs = imgs[split_1:split_2]
    test_imgs = imgs[split_2:]

    # Normalizing the images
    train_imgs = train_imgs / 255.0
    test_imgs = test_imgs / 255.0
    val_imgs = val_imgs / 255.0

    labels = np.load('75/labels.npy')
    labels = labels.astype('int32')
    labels = labels[indices]
    train_labels = labels[:split_1]
    val_labels = labels[split_1:split_2]
    test_labels = labels[split_2:]

    train_imgs = train_imgs.reshape((train_imgs.shape[0], 75, 75, 1))
    val_imgs = val_imgs.reshape((val_imgs.shape[0], 75, 75, 1))
    test_imgs = test_imgs.reshape((test_imgs.shape[0], 75, 75, 1))


## Classification Model

In [33]:
# Convert the time into 24 separate labels
def conv_time_24(time):
    ntime = 0
    if time[1] > 30:
        ntime = (time[0] + 0.5)
    else:
        ntime = time[0]
    return ntime

# Convert the time into 720 separate labels 
def conv_time_720(time):
    return time[0]*60 + time[1]

conv_time = conv_time_720
while True:
    initialize()
    train_labels_converted = np.array([conv_time(time) for time in train_labels])
    test_labels_converted = np.array([conv_time(time) for time in test_labels])
    val_labels_converted = np.array([conv_time(time) for time in val_labels])

    encoder = LabelEncoder()
    test_labels_encoded = encoder.fit_transform(test_labels_converted.reshape(-1))
    train_labels_encoded = encoder.fit_transform(train_labels_converted.reshape(-1))
    val_labels_encoded = encoder.fit_transform(val_labels_converted.reshape(-1))

    OHencoder = OneHotEncoder(sparse_output=False)
    train_labels_oh = OHencoder.fit_transform(train_labels_encoded.reshape(-1, 1))
    val_labels_oh = OHencoder.fit_transform(val_labels_encoded.reshape(-1, 1))
    print(val_labels_encoded)
    print(val_labels_oh)
    
    # Check if all labels are present in the validation set, if not, reshuffle
    try:
        val_labels_oh = val_labels_oh.reshape((val_labels_oh.shape[0], 720))
        break
    except:
        pass

[403 524 233 ... 371 157  71]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[136 363 378 ... 149 508 181]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[289 258 313 ... 448 303 522]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[657  78 349 ... 426 434 656]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[604 672 490 ... 435 708  55]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[112 350 263 ... 428  31 460]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 

In [30]:
#Common sense error function- checks the smaller of the differences by converting to military time
def common_sense_error(true, pred):
    true = K.cast(true, 'float32')
    diff1 = K.abs(pred-true)
    diff2 = K.abs(pred+12-true)
    return K.minimum(diff1, diff2)

In [ ]:
input_shape = (75, 75, 1)
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(kernel_size=(3,3), strides = (2,2), activation="relu", filters=32))
model.add(keras.layers.Conv2D(activation="relu", filters=32, kernel_size=(3,3), input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))

model.add(keras.layers.Flatten())
# First layer - one neuron for each pixel
model.add(keras.layers.Dense(units=625, activation="relu"))
model.add(keras.layers.Dense(units=512, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dense(units=720, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[common_sense_error])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(train_imgs, train_labels_oh, epochs=10, batch_size=256, validation_data=(val_imgs, val_labels_oh), callbacks=[early_stop])

Epoch 1/10


/vol/home/s4201388/Documents/Leiden/IDL/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - common_sense_error: 0.0028 - loss: 6.5793 - val_common_sense_error: 0.0028 - val_loss: 6.5794
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - common_sense_error: 0.0028 - loss: 6.5792 - val_common_sense_error: 0.0028 - val_loss: 6.5795
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - common_sense_error: 0.0028 - loss: 6.5791 - val_common_sense_error: 0.0028 - val_loss: 6.5797
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - common_sense_error: 0.0028 - loss: 6.5791 - val_common_sense_error: 0.0028 - val_loss: 6.5799
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - common_sense_error: 0.0028 - loss: 6.5789 - val_common_sense_error: 0.0028 - val_loss: 6.5803
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - common_sense_error: 0.0028 - loss: 6.5789 - val_common_sense_error: 0.0028 - val_loss: 6.5816
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - common_sense_error: 0.0028 - loss: 6.5785 - val_common_sense_error: 0.0028 -

In [35]:
# Measuring accuracy- picking the highest confidence label and comparing it to the true label
preds = model.predict(test_imgs)
preds = np.argmax(preds, axis=1)

results = encoder.inverse_transform(preds)
accuracy = np.sum(results == test_labels_converted) / len(test_labels_converted)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
0.16666666666666669 %


## Regression Model

In [82]:
import tensorflow.keras.backend as K

#returning the common sense difference between two times
def common_sense_error(true, pred):
    true = K.cast(true, 'float32')
    diff_1 = K.abs(true - pred)
    diff_2 = K.abs(true - (pred + 12))

    return K.minimum(diff_1, diff_2)

In [ ]:
initialize()

# Convert the time into a continuous value
def conv_time(time):
    return round(time[0] + time[1]/60, 3)

train_labels_reg = np.array([conv_time(time) for time in train_labels])
test_labels_reg = np.array([conv_time(time) for time in test_labels])
val_labels_reg = np.array([conv_time(time) for time in val_labels])

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(activation='relu', filters=32, kernel_size=(3,3), input_shape=(75, 75, 1)))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.Conv2D(filters=32 ,kernel_size=(3,3), activation='relu'))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=512, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=1, activation="softplus"))
model.compile(loss="mse", optimizer="adam", metrics=[common_sense_error])

early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)

model.fit(train_imgs, train_labels_reg, epochs=10, batch_size = 512, validation_data = (val_imgs, val_labels_reg), callbacks = [early_stop])

Epoch 1/10


/vol/home/s4201388/Documents/Leiden/IDL/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 223ms/step - common_sense_error: 3.0834 - loss: 19.0580 - val_common_sense_error: 3.0621 - val_loss: 12.8423
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 259ms/step - common_sense_error: 3.0335 - loss: 12.6101 - val_common_sense_error: 2.9572 - val_loss: 11.6659
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 257ms/step - common_sense_error: 3.0004 - loss: 12.2394 - val_common_sense_error: 2.9553 - val_loss: 12.1154
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 259ms/step - common_sense_error: 3.0051 - loss: 12.0379 - val_common_sense_error: 3.0384 - val_loss: 11.6103
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 257ms/step - common_sense_error: 3.0441 - loss: 11.8143 - val_common_sense_error: 2.9587 - val_loss: 11.4221
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 261ms/step - common_sense_error: 3.0263 - loss: 10.8949 - val_common_sense_error: 3.0077 - val_loss: 10.6288
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 263ms/step - common_sense_error: 3.1190 - loss: 10.6135 - val_common_s

In [ ]:
# Measuring accuracy within 1 minute of the true time
reg_preds = model.predict(test_imgs)
accuracy = np.mean(abs(reg_preds - test_labels_reg) < 0.16)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
2.6464506172839504 %


## Two-Headed Model

In [38]:
initialize()

train_hours = train_labels[:, 0]
train_minutes = train_labels[:, 1]

val_hours = val_labels[:, 0]
val_minutes = val_labels[:, 1]

test_hours = test_labels[:, 0]
test_minutes = test_labels[:, 1]

train_minutes = train_minutes / 60
test_minutes = test_minutes / 60
val_minutes = val_minutes / 60

In [17]:
import tensorflow.keras.backend as K

def common_sense_hour(true, pred):
    true = K.cast(true, 'float32')
    diff1 = K.abs(pred-true)
    diff2 = K.abs(pred+12-true)
    return K.minimum(diff1, diff2)

def common_sense_minute(true, pred):
    true = K.cast(true, 'float32')
    diff1 = K.abs(pred-true)
    diff2 = K.abs(pred+60-true)
    return K.minimum(diff1, diff2)

In [40]:
inp = keras.layers.Input(shape = (75,75,1))
model = keras.layers.Convolution2D(32,kernel_size = (5,5), strides= (2,2), activation = "relu")(inp)
model = keras.layers.MaxPooling2D(pool_size =2)(model)
model = keras.layers.Convolution2D(32,kernel_size = (3,3),activation = "relu")(model)
model = keras.layers.Convolution2D(32,kernel_size = (3,3),activation = "relu")(model)
model = keras.layers.MaxPooling2D(pool_size =2)(model)
model = keras.layers.Convolution2D(64,kernel_size = (3,3),activation = "relu")(model)
model = keras.layers.Convolution2D(64,kernel_size = (1,1),activation = "relu")(model)
model = keras.layers.Flatten()(model)

d = keras.layers.Dense(256,activation = "relu")(model)
d = keras.layers.Dense(256,activation = "relu")(d)
d = keras.layers.Dropout(0.1)(d)
d = keras.layers.Dense(256,activation = "relu")(d)

# The hour head predicts confidence amonst 12 label classes- one for each hour 
hour = keras.layers.Dense(256,activation = "relu")(d)
hour = keras.layers.Dense(128,activation = "relu")(hour)
hour = keras.layers.Dense(64,activation = "relu")(hour)
hour = keras.layers.Dense(32,activation = "relu")(hour)
hour = keras.layers.Dense(16,activation = "relu")(hour)
hour = keras.layers.Dense(12,activation= "softmax", name= "hour")(hour)

# The minute head runs a regression on a continuous value
minute = keras.layers.Dense(256,activation = "relu")(d)
minute = keras.layers.Dense(256,activation = "relu")(minute)
minute = keras.layers.Dense(256,activation = "relu")(minute)
minute = keras.layers.Dense(128,activation = "relu")(minute)
minute = keras.layers.Dense(64,activation = "relu")(minute)
minute = keras.layers.Dense(32,activation = "relu")(minute)
minute = keras.layers.Dense(16,activation = "relu")(minute)
minute = keras.layers.Dense(1, activation = "softplus", name = "minute")(minute)

model = tf.keras.models.Model(inputs=inp, outputs=[hour, minute])
optim = tf.keras.optimizers.Adam()
model.compile(loss=['sparse_categorical_crossentropy', 'mse'], optimizer=optim, metrics=['accuracy',"mae"])

early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)

model.fit(train_imgs, [train_hours, train_minutes], epochs=30, batch_size = 512, validation_data = (val_imgs, [val_hours, val_minutes]), callbacks = [early_stop])

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - hour_accuracy: 0.0877 - hour_loss: 2.4857 - loss: 2.5919 - minute_loss: 0.1062 - minute_mae: 0.2738 - val_hour_accuracy: 0.0781 - val_hour_loss: 2.4853 - val_loss: 2.5691 - val_minute_loss: 0.0831 - val_minute_mae: 0.2513
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - hour_accuracy: 0.0821 - hour_loss: 2.4849 - loss: 2.5689 - minute_loss: 0.0840 - minute_mae: 0.2512 - val_hour_accuracy: 0.0800 - val_hour_loss: 2.4854 - val_loss: 2.5690 - val_minute_loss: 0.0829 - val_minute_mae: 0.2513
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - hour_accuracy: 0.0903 - hour_loss: 2.4848 - loss: 2.5692 - minute_loss: 0.0844 - minute_mae: 0.2511 - val_hour_accuracy: 0.0794 - val_hour_loss: 2.4859 - val_loss: 2.5700 - val_minute_loss: 0.0844 - val_minute_mae: 0.2519
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - hour_accuracy: 0.0829 - hour_loss: 2.4849 - loss: 2.5684 - minute_loss: 0.0835 - minute_mae: 0.2496 - val_hour_accuracy: 0

In [22]:
predictions = model.predict(test_imgs)
hour_p = np.argmax(predictions[0], axis = 1)
minutes_p = predictions[1]

accuracy = np.mean(np.abs(hour_p - test_hours) < 1) * np.mean(np.abs(minutes_p - test_minutes) < 5)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
7.222222222222221 %
